In [93]:
from geopy.geocoders import Nominatim
import ssl
import certifi
import folium
import requests
import numpy as np
import json

# Initialize the geolocator with a user agent and custom SSL context
geolocator = Nominatim(user_agent="medical_locator", ssl_context=ssl.create_default_context(cafile=certifi.where()),
    timeout=50)

# Define the location and search queries
location = 'Edinburgh'
search_queries = ['Medical', 'Hospital', 'Clinic', 'Healthcare', 'Health Centre', 'Medical Practice']

# Create a map centered around Edinburgh
edinburgh_coords = [55.9533, -3.1883]
map_ = folium.Map(location=edinburgh_coords, zoom_start=13)

# List to store the coordinates of the hospitals
hospital_coords = [edinburgh_coords]

# Perform the search for each query and plot the results
locations_coord = [[-3.1883,55.9533]]
nodes = ['lab']
coords_dic = {'lab': edinburgh_coords}
i = 0
for query in search_queries:
    results = geolocator.geocode(f"{query}, {location}", exactly_one=False, limit=50)
    if results:
        for result in results:
            if i <=20:
                i += 1
                name = result.address
                lat = result.latitude
                lng = result.longitude
                locations_coord.append([lng,lat])
                location_name = name.split(',')[0]
                nodes.append(location_name)
                print(f"{location_name}: {lat}, {lng}")
                hospital_coords.append([lat, lng])
                coords_dic[location_name] = [lat,lng]
                folium.Marker([lat, lng], popup=name).add_to(map_)
folium.Marker(edinburgh_coords, popup='laboratory',icon=folium.Icon(color='red')).add_to(map_)
nodes.append('lab1')
hospital_coords.append(edinburgh_coords)
locations_coord.append([-3.1883,55.9533])
coords_dic['lab1'] = edinburgh_coords

# Connect the first and the last hospital locations with a line
if len(hospital_coords) >= 2:
    for index, value1 in enumerate(hospital_coords):
        for index, value2 in enumerate(hospital_coords):
            if value1 != value2:
                folium.PolyLine(locations=[value1,value2], color='blue').add_to(map_)

# Display the map (this works in Jupyter notebooks)
map_

# locations_coord.append([-3.1883,55.9533])
# Hospital: 55.948966049999996, -3.201482638965784
# Atos Healthcare: 55.9465497, -3.201035
# Eyre Medical Practice: 55.9614472, -3.1995521
Boroughloch Medical Practice: 55.9412785, -3.1772047


coords = []
coords.append([-3.1883,55.9533])
coords.append([-3.201482638965784,55.948966049999996])
coords.append([-3.201035,55.9465497])
coords.append([-3.1995521,55.9614472])
coords.append([-3.17720473,55.9412785])

Hospital: 55.948966049999996, -3.201482638965784
Mountcastle HealthCare Centre: 55.940432, -3.11628463182614
Atos Healthcare: 55.9465497, -3.201035
Chinese Medicine & Healthcare: 55.9760082, -3.1677397
Cramond Medical Practice: 55.9755937, -3.2989659
Gilmore Medical Practice: 55.9439488, -3.2056762
Southfield Medical Practice: 55.94037685, -3.1172157813591186
Marchmont Medical Practice: 55.9392865, -3.1963919
Hermitage Medical Practice: 55.9258449, -3.2085068
Morningside Medical Practice: 55.9302081, -3.2099997
Durham Road Medical Practice: 55.9471375, -3.1197208
Colinton Medical Practice: 55.9088917, -3.252190111207338
Dalkeith Road Medical Practice: 55.9349585, -3.1681302
Bruntsfield Medical Practice: 55.93562755000001, -3.207100793805742
Leven Medical Practice: 55.9438384, -3.2055487
Eyre Medical Practice: 55.9614472, -3.1995521
Ratho Medical Practice: 55.9206564, -3.3879940628676177
Murrayfield Medical Practice: 55.9451118, -3.2432691
Meadows Medical Practice: 55.9424297, -3.200718

In [113]:
# Lab [-3.1883,55.9533]
# Atos Healthcare: 55.9465497, -3.201035
# Mountcastle HealthCare Centre: 55.940432, -3.11628463182614
# Chinese Medicine & Healthcare: 55.9760082, -3.1677397
# Morningside Medical Practice: 55.9302081, -3.2099997


coords = []
coords.append([-3.1883,55.9533]) #lab
coords.append([-3.201035,55.9465497])
coords.append([-3.11628463182614,55.940432])
coords.append([-3.1677397,55.9760082])

body = {"locations":coords,"metrics":["duration","distance"],"units":"km"}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf62481fcbd26371b3498fb066830493c5e1f8',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
data = json.loads(call.text)

print(call.status_code, call.reason)
print(call.text)

# Convert distances and durations to NumPy arrays
distances = np.array(data['distances'])
durations = np.array(data['durations'])/60
durations = np.round(durations, 0)

print(durations)

200 OK
{"durations":[[0.0,225.14,727.85,552.08,551.71],[262.77,0.0,876.56,700.79,342.15],[700.74,881.67,0.0,586.53,936.95],[525.02,705.95,587.06,0.0,1032.52],[587.85,340.66,917.77,1025.87,0.0]],"distances":[[0.0,1.54,5.64,3.74,3.6],[1.72,0.0,6.86,4.96,2.14],[5.57,6.88,0.0,5.7,8.03],[3.67,4.98,5.71,0.0,7.04],[3.85,2.21,7.95,7.09,0.0]],"destinations":[{"location":[-3.188528,55.953389],"snapped_distance":17.31},{"location":[-3.201028,55.946395],"snapped_distance":17.23},{"location":[-3.116023,55.940446],"snapped_distance":16.34},{"location":[-3.167795,55.976036],"snapped_distance":4.61},{"location":[-3.209751,55.930225],"snapped_distance":15.63}],"sources":[{"location":[-3.188528,55.953389],"snapped_distance":17.31},{"location":[-3.201028,55.946395],"snapped_distance":17.23},{"location":[-3.116023,55.940446],"snapped_distance":16.34},{"location":[-3.167795,55.976036],"snapped_distance":4.61},{"location":[-3.209751,55.930225],"snapped_distance":15.63}],"metadata":{"attribution":"openroutes

In [156]:
# Create a map centered around Edinburgh
coords = []
coords.append([-3.1883,55.9533]) #lab
coords.append([-3.201035,55.9465497])
coords.append([-3.11628463182614,55.940432])
coords.append([-3.1677397,55.9760082])

edinburgh_coords = [55.9533, -3.1883]
map_ = folium.Map(location=edinburgh_coords, zoom_start=13)

folium.Marker([coords[0][1], coords[0][0]], popup='laboratory',icon=folium.Icon(color='red')).add_to(map_)
print(coords)
for coord in coords[1:]:
    print(coord[0],coord[1])
    folium.Marker([coord[1], coord[0]], popup=name).add_to(map_)

for coord1 in coords:
    for coord2 in coords:
        folium.PolyLine(locations=[[coord1[1],coord1[0]],[coord2[1],coord2[0]]], color='blue').add_to(map_)

map_

[[-3.1883, 55.9533], [-3.201035, 55.9465497], [-3.11628463182614, 55.940432], [-3.1677397, 55.9760082]]
-3.201035 55.9465497
-3.11628463182614 55.940432
-3.1677397 55.9760082


# The NeurADP

In [66]:
import states as st
import numpy as np
import scenarios_generator as sg
import neuradp as ndp
import importlib
import replay_buffer as rb
importlib.reload(st)
importlib.reload(sg)
importlib.reload(ndp)
importlib.reload(rb)
import copy
import pandas as pd

from geopy.geocoders import Nominatim
import requests
import numpy as np
import json


rates = [5,10,15,20,25,30]
for rate in rates:

    #result collector
    x_realization = np.array([])
    y_total_duration = np.array([])

    replay_buffer = rb.ReplayBuffer()
    distance_array = np.array([])
    M = []

    log = {}
    expired = []
    st.state.restart_nn()

    centres = {}
    for i in range(1,4):
        centres['centre_'+ str(i)] = st.Centre(i)

    vehicles = {}
    for i in range(1,4):
        vehicles['vehicle_' + str(i)] = st.Vehicle(str(i),0,centres,type='Private')

    for i in centres.keys():
        vehicles['vehicle_' + str(i)] = st.Vehicle(str(i),0,centres,type='External')

    def calculate_weight_distance(model1, model2):
        distance = 0.0
        for layer1, layer2 in zip(model1.weights, model2.weights):
            distance += np.linalg.norm(layer1.numpy() - layer2.numpy())
        return distance

    def new_duration_matrix(centres):
        coords = []
        coords.append([-3.1883,55.9533]) #lab
        coords.append([-3.201035,55.9465497])
        coords.append([-3.11628463182614,55.940432])
        coords.append([-3.1677397,55.9760082])
        coords.append([-3.2099997,55.9302081])

        body = {"locations":coords[:len(centres)+1],"metrics":["duration","distance"],"units":"km"}
        headers = {
            'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
            'Authorization': '5b3ce3597851110001cf62481fcbd26371b3498fb066830493c5e1f8',
            'Content-Type': 'application/json; charset=utf-8'
        }
        call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
        data = json.loads(call.text)


        # Convert distances and durations to NumPy arrays
        distances = np.array(data['distances'])
        durations = np.array(data['durations'])/60
        durations = np.round(durations, 0)

        nodes = ['lab'] + [c for c in centres.keys()]
        duration = {}

        for i, node1 in enumerate(nodes):
            for j,node2 in enumerate(nodes):
                if node1 != node2:
                    duration[node1,node2] = durations[i,j]
        return duration


    def duration_matrix(centres):
        nodes = [i for i,j in centres.items()]
        nodes.insert(0,'lab')
        nodes.append('lab')
        nodes

        duration = {}

        m = 16384
        a = 4781
        c = 8521
        z = 1136
        k = 0

        for i in nodes[:-1]:
            for j in nodes[1:]:
                if i != j:
                    k += 1
                    z = (a*z + c) % m
                    u = z/m
                    duration[i,j] = round((30-10)*u + 10,0)

        return duration

    duration = new_duration_matrix(centres)

    sg.sample_path.restart()
    scenarios = {}

    M = []

    for scenario in range(1,200):
        for key,value in centres.items():
            scenarios[scenario,key] = sg.sample_path(key,scenario,rate,7)

    first_try_state = st.state('st1',vehicles,centres,duration,epoch_interval=5)
    first_try_state.time = 480

    with open(f'output_{rate}_LR.txt', 'w') as f:
        myopic = True
        for realization in range(1,200):
            print(f'realization: {realization}')
            f.write(f'realization: {realization}\n')
            st.state.restart_global_attributes()
            first_try_state.restart_state()
            first_try_state.realization = realization
            first_try_state.time = 480
            for transition in range(480,960,first_try_state.epoch_interval):
                spre_transition = copy.deepcopy(first_try_state)
                M.append(copy.deepcopy(first_try_state))
                f.write(f'pre: {first_try_state}\n')
                training = True
                first_try_state.state_update_pos(myop=myopic,train=training)
                spos_transition = copy.deepcopy(first_try_state)
                reward = spos_transition.immediate_cost
                f.write(f'pos: {first_try_state}\n')
                f.write(f'actions: {first_try_state.action}\n')
                first_try_state.state_update_pre(transition,scenarios,realization)
                experience = st.Experience(spre_transition,spos_transition,reward,None,transition,None,realization)
                replay_buffer.add(experience)
                if replay_buffer.size == 1500:
                    myopic = False
                if replay_buffer.size > 1500 and realization < 100:
                    if transition % 50== 0:
                        st.state.nnmodel.soft_update(replay_buffer)

            print(f'total_duration is {first_try_state.total_duration}')
            f.write(f'total_duration is {first_try_state.total_duration}\n')
            print(f'buffer size: {replay_buffer.size}')
            print(f'weight distance is {calculate_weight_distance(st.state.nnmodel.target, st.state.nnmodel.model)}')
            
            if realization > 10:
                dummy_feat = M[6].feature_nn('vehicle_1').reshape(1,-1)
                result_target = st.state.nnmodel.target.predict(dummy_feat,verbose=0)
                result_model = st.state.nnmodel.model.predict(dummy_feat,verbose=0)
                print(f'prediction model: {result_model}, prediction target: {result_target}')
            print(f'myopic: {myopic}')
            x_realization = np.append(x_realization,realization)
            y_total_duration = np.append(y_total_duration,first_try_state.total_duration)

        df = pd.DataFrame({
            'Day': x_realization,
            'Value': y_total_duration
        })

        df.to_excel(f'LR_{rate}min_specimen3V3C.xlsx', index=False)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-18
realization: 1
total_duration is 142.0
buffer size: 96
weight distance is 0.6392425894737244
myopic: True
realization: 2
total_duration is 150.0
buffer size: 191
weight distance is 0.6392425894737244
myopic: True
realization: 3
total_duration is 142.0
buffer size: 286
weight distance is 0.6392425894737244
myopic: True
realization: 4
total_duration is 100.0
buffer size: 381
weight distance is 0.6392425894737244
myopic: True
realization: 5
total_duration is 130.0
buffer size: 471
weight distance is 0.6392425894737244
myopic: True
realization: 6
total_duration is 132.0
buffer size: 565
weight distance is 0.6392425894737244
myopic: True
realization: 7
total_duration is 142.0
buffer size: 659
weight distance is 0.6392425894737244
myopic: True
realization: 8
total_duration is 108.0
buffer size: 751
weight distance is 0.6392425894737244
myopic: True
realization: 9
total_duration is 142.0
buffer size: 83

# MIP Under Perfect Information

In [328]:
import states as st
import scenarios_generator as sg
import neuradp as ndp
import importlib
import replay_buffer as rb
importlib.reload(st)
importlib.reload(sg)
importlib.reload(ndp)
importlib.reload(rb)
import copy
import pandas as pd

import mip_upi as mip
import numpy as np
importlib.reload(mip)

from geopy.geocoders import Nominatim
import ssl
import certifi
import folium
import requests
import numpy as np
import json


# rate = 20
rates = [5]
for rate in rates:

    #result collector
    x_realization = np.array([])
    y_objective = np.array([])
    y_runtime = np.array([])
    mip_gap = np.array([])

    replay_buffer = rb.ReplayBuffer()
    distance_array = np.array([])
    M = []

    log = {}
    expired = []
    st.state.restart_nn()

    centres = {}
    for i in range(1,4):
        centres['centre_'+ str(i)] = st.Centre(i)

    vehicles = {}
    for i in range(1,4):
        vehicles['vehicle_' + str(i)] = st.Vehicle(str(i),0,centres,type='Private')

    for i in centres.keys():
        vehicles['vehicle_' + str(i)] = st.Vehicle(str(i),0,centres,type='External')

    def calculate_weight_distance(model1, model2):
        distance = 0.0
        for layer1, layer2 in zip(model1.weights, model2.weights):
            distance += np.linalg.norm(layer1.numpy() - layer2.numpy())
        return distance

    def new_duration_matrix(centres):
        coords = []
        coords.append([-3.1883,55.9533]) #lab
        coords.append([-3.201035,55.9465497])
        coords.append([-3.11628463182614,55.940432])
        coords.append([-3.1677397,55.9760082])
        coords.append([-3.2099997,55.9302081])

        body = {"locations":coords[:len(centres)+1],"metrics":["duration","distance"],"units":"km"}
        headers = {
            'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
            'Authorization': '5b3ce3597851110001cf62481fcbd26371b3498fb066830493c5e1f8',
            'Content-Type': 'application/json; charset=utf-8'
        }
        call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
        data = json.loads(call.text)

        # print(call.status_code, call.reason)
        # print(call.text)

        # Convert distances and durations to NumPy arrays
        distances = np.array(data['distances'])
        durations = np.array(data['durations'])/60
        durations = np.round(durations, 0)

        nodes = ['lab'] + [c for c in centres.keys()]
        duration = {}

        for i, node1 in enumerate(nodes):
            for j,node2 in enumerate(nodes):
                if node1 != node2:
                    duration[node1,node2] = durations[i,j]
        return duration


    def duration_matrix(centres):
        nodes = [i for i,j in centres.items()]
        nodes.insert(0,'lab')
        nodes.append('lab')
        nodes

        duration = {}

        m = 16384
        a = 4781
        c = 8521
        z = 1136
        k = 0

        for i in nodes[:-1]:
            for j in nodes[1:]:
                if i != j:
                    k += 1
                    z = (a*z + c) % m
                    u = z/m
                    duration[i,j] = round((30-10)*u + 10,0)

        return duration

    # duration = new_duration_matrix(centres)

    sg.sample_path.restart()
    scenarios = {}

    M = []

    for scenario in range(1,200):
        for key,value in centres.items():
            scenarios[scenario,key] = sg.sample_path(key,scenario,rate,7)




    for realization in range(100,200):
        print(f'realization {realization}')
        x_realization = np.append(x_realization,realization)
        objective, runtime, gap = mip.mip_upi(vehicles,centres,duration,scenarios,realization)
        y_objective = np.append(y_objective,objective)
        y_runtime = np.append(y_runtime,runtime)
        mip_gap = np.append(mip_gap,gap)

    df = pd.DataFrame({
            'Day': x_realization,
            'Value': y_objective,
            'Gap' : mip_gap,
            'Runtime' : y_runtime
        })

    df.to_excel(f'UPI_{rate}min_specimen3V3C.xlsx', index=False)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-18
Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-18
realization 100
Set parameter TimeLimit to value 240
Solution is not optimal.
Best Obj: 117
MIP Gap: 0.666667
realization 101
Set parameter TimeLimit to value 240
Solution is not optimal.
Best Obj: 117
MIP Gap: 0.666667
realization 102
Set parameter TimeLimit to value 240
Solution is not optimal.
Best Obj: 117
MIP Gap: 0.649573
realization 103
Set parameter TimeLimit to value 240
Solution is not optimal.
Best Obj: 117
MIP Gap: 0.623932
realization 104
Set parameter TimeLimit to value 240
Solution is not optimal.
Best Obj: 117
MIP Gap: 0.666667
realization 105
Set parameter TimeLimit to value 240
Solution is not optimal.
Best Obj: 117
MIP Gap: 0.649573
realization 106
Set parameter TimeLimit to value 240
Solution is not optimal.
Best Obj: 117
MIP Gap: 0.666667
realization 107
Set parameter TimeLimit to value

# Deterministic Approach

In [408]:
import gurobipy as gp
from gurobipy import GRB
import os
import math

import requests
import json
import numpy as np


import states as st
import numpy as np
import scenarios_generator as sg
import neuradp as ndp
import importlib
import replay_buffer as rb
importlib.reload(st)
importlib.reload(sg)
importlib.reload(ndp)
importlib.reload(rb)
import copy


os.environ['GRB_LICENSE_FILE'] = 'c9697abb-f79e-46a9-b050-2895ccc98a02'
env = gp.Env(empty=True)
env.start()

centres = {}
for i in range(1,4):
    centres['centre_'+ str(i)] = st.Centre(i)

vehicles = {}
for i in range(1,4):
    vehicles['vehicle_' + str(i)] = st.Vehicle(str(i),0,centres,type='Private')



# Parameters
n = len(centres)
o = 60*8
e = 480
l = 500
time_limit_after_closing = 20
maximum_time_allowed = 60
t_maximum_travel_duration = 120
t_g = 150
p_g = math.ceil(o/t_g)

def new_duration_matrix(centres):
    coords = []
    coords.append([-3.1883,55.9533]) #lab
    coords.append([-3.201035,55.9465497])
    coords.append([-3.11628463182614,55.940432])
    coords.append([-3.1677397,55.9760082])
    coords.append([-3.2099997,55.9302081])

    body = {"locations":coords[:len(centres)+1],"metrics":["duration","distance"],"units":"km"}
    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': '5b3ce3597851110001cf62481fcbd26371b3498fb066830493c5e1f8',
        'Content-Type': 'application/json; charset=utf-8'
    }
    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
    data = json.loads(call.text)

    # print(call.status_code, call.reason)
    # print(call.text)

    # Convert distances and durations to NumPy arrays
    distances = np.array(data['distances'])
    durations = np.array(data['durations'])/60
    durations = np.round(durations, 0)

    nodes = ['lab'] + [c for c in centres.keys()]
    duration = {}

    for i, node1 in enumerate(nodes):
        for j,node2 in enumerate(nodes):
            if node1 != node2:
                duration[node1,node2] = durations[i,j]
    return duration

# duration = new_duration_matrix(centres)
duration = {('lab', 'centre_1'): 4.0,
 ('lab', 'centre_2'): 12.0,
 ('lab', 'centre_3'): 9.0,
 ('centre_1', 'lab'): 4.0,
 ('centre_1', 'centre_2'): 15.0,
 ('centre_1', 'centre_3'): 12.0,
 ('centre_2', 'lab'): 12.0,
 ('centre_2', 'centre_1'): 15.0,
 ('centre_2', 'centre_3'): 10.0,
 ('centre_3', 'lab'): 9.0,
 ('centre_3', 'centre_1'): 12.0,
 ('centre_3', 'centre_2'): 10.0}

# Sets
# Set of nodes V
centres_set = [i for i in centres.keys()]
request = {}
for cent in centres.keys():
    request[cent] = [str(cent)+'_req_'+str(i+1) for i in range(p_g)]


nodes = ['lab']
for key,values in request.items():
    for v in values:
        nodes.append(v)

pre_arcs = {}
for i in nodes:
    for j in nodes:
        name_i = i.rsplit('_',2)[0] if i != 'lab' else 'lab'
        name_j = j.rsplit('_',2)[0] if j != 'lab' else 'lab'
        if name_i != name_j:
            pre_arcs[i,j] = duration[name_i,name_j]

arcs, durations = gp.multidict(pre_arcs)

index = [i for i,n in enumerate(nodes)]

with gp.Model(env=env) as m:
    # Create variables
    travel = m.addVars(arcs,vtype=GRB.BINARY,name='travel')
    visit = m.addVars(nodes,name='visit_time')
    duration = m.addVars(nodes[1:],name='duration')
    remaining = m.addVars(nodes[1:],name='remaining')
    opening = m.addVars(centres_set,name='opening')
    closing = m.addVars(centres_set,name='closing')

    # Set Objective Function
    obj = gp.LinExpr()
    for i in nodes[1:]:
        obj += duration[i]
    m.setObjective(obj, GRB.MINIMIZE)

    # Constraints
    c1 = m.addConstrs((travel.sum(nodes,j) - travel.sum(j,nodes) == 0 for j in nodes),name='c1') #ok
    c2 = m.addConstrs((travel.sum(nodes,j) == 1 for j in nodes[1:]),name='c2') #ok
    c3_1 = m.addConstrs((opening[centre] >= e for centre in centres_set),name='c3_1') #ok
    c3_2 = m.addConstrs((opening[centre] <= l for centre in centres_set),name='c3_2') #ok
    c4 = m.addConstrs((opening[centre] + o == closing[centre] for centre in centres_set),name='c4') #ok
    c5 = m.addConstrs((visit[j] >= visit[i] + 1 + durations[i,j] - 100000*(1 - travel[i,j]) for i in nodes for j in nodes[1:] if (i.rsplit('_',2)[0] if i != 'lab' else 'lab') != (j.rsplit('_',2)[0] if j != 'lab' else 'lab')),name='c5') #ok
    c6 = m.addConstrs((visit[request[centre][0]] - opening[centre] <= t_g for centre in centres_set),name='c6') #change this for new problem def
    c7 = m.addConstrs((visit[request[centre][len(request[centre])-i-1]] - visit[request[centre][len(request[centre])-i-2]] <= t_g for centre in centres_set for i in range(len(request[centre])-1)),name='c7')
    c8_1 = m.addConstrs((visit[request[centre][-1]] >=  closing[centre] for centre in centres_set),name='c8_1') #ok
    c8_2 = m.addConstrs((visit[request[centre][-1]] <=  closing[centre] + time_limit_after_closing for centre in centres_set),name='c8_2') #ok
    c9 = m.addConstrs((t_maximum_travel_duration - remaining[i] + 100000*(1-travel[i,'lab']) >= durations[i,'lab'] for i in nodes[1:] ) ,name='c9') #ok
    c10 = m.addConstrs((remaining[j] - remaining[i] + 100000*(1 - travel[i,j]) >= visit[j] - visit[i] for i in nodes[1:] for j in nodes[1:] if (i.rsplit('_',2)[0] if i != 'lab' else 'lab') != (j.rsplit('_',2)[0] if j != 'lab' else 'lab')),name='c10') #ok
    c11 = m.addConstrs((remaining[i] >= 0 for i in nodes[1:]),name='c11') #ok
    # c11 = m.addConstrs((remaining[request[centre][0]] - (visit[request[centre][0]] - opening[centre]) >= 0 for centre in centres_set),name='c11')
    # c12 = m.addConstrs((remaining[request[centre][len(request[centre])-i-1]] - (visit[request[centre][len(request[centre])-i-1]] - visit[request[centre][len(request[centre])-i-2]]) >= 0 for centre in centres_set for i in range(len(request[centre])-1)),name='c12')
    c12 = m.addConstrs((closing[centre]-visit[request[centre][-1]] <= t_g for centre in centres_set),name='c12')
    c13 = m.addConstrs((duration[i] >= t_maximum_travel_duration - remaining[i] + durations['lab',i]*travel['lab',i] - 100000*(1-travel['lab',i]) for i in nodes[1:]),name='c13') #ok

    m.optimize()
    m.write('matheuristic.lp')
    if m.Status == GRB.OPTIMAL:
        visit_time = m.getAttr('X',visit)
        result_dict = {centre: [] for centre in centres_set}
        print(f"Objective value: {m.objVal}")
        for v in m.getVars():
            if v.x > 0:
                print(f"{v.varName} = {v.x}")
        for key, values in request.items():
            for v in request[key]:
                if visit_time[v] > 0:
                    result_dict[key].append(visit_time[v])

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-18
Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-18
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 295 rows, 163 columns and 1278 nonzeros
Model fingerprint: 0xb7b625d2
Variable types: 43 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Found heuristic solution: objective 1147.0000000
Presolve removed 108 rows and 40 columns
Presolve time: 0.01s
Presolved: 187 rows, 123 columns, 1026 nonzeros
Variable types: 39 continuous, 84 integer (84 binary)

Root relaxation: objective 0.000000e+00, 51 iterat